# loading

In [8]:
import os
import struct
import random
import numpy as np
import matplotlib.pyplot as plt

def load_mnist(path, kind='train'):
    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte'
                                % kind)
    images_path = os.path.join(path,
                               '%s-patterns-idx3-ubyte'
                               % kind)

    with open(labels_path, 'rb') as lbpath:
        magic, n = struct.unpack('>II',
                                 lbpath.read(8))
        labels = np.fromfile(lbpath,
                             dtype=np.uint8)

    with open(images_path, 'rb') as imgpath:
        magic, num, rows, cols = struct.unpack(">IIII",
                                               imgpath.read(16))
        images = np.fromfile(imgpath,
                             dtype=np.uint8).reshape(len(labels), 784)

    return images, labels

#### Loading the data

X_train, y_train = load_mnist('./data', kind='train')
X_test, y_test = load_mnist('./data', kind='test')

In [9]:
def load_mnist(path, kind='train'):
    """Load MNIST data from `path`"""
    images_path = os.path.join(path,
                               '%s-patterns-idx3-ubyte'
                               % kind)

    with open(images_path, 'rb') as imgpath:
        magic, num, rows, cols = struct.unpack(">IIII",
                                               imgpath.read(16))
        images = np.fromfile(imgpath,
                             dtype=np.uint8).reshape(60000, 784)

    return images

X_testall = load_mnist('./data', kind='testall')

FileNotFoundError: [Errno 2] No such file or directory: './data\\testall-patterns-idx3-ubyte'

In [10]:
print('train_sample_number:\t:%d, column_number:%d' %(X_train.shape[0], X_train.shape[1]))
print('test_sample_number:\t:%d, column_number:%d' %(X_test.shape[0], X_test.shape[1]))
print('testall_sample_number\t:%d, column_number:%d' %(X_testall.shape[0], X_testall.shape[1]))

train_sample_number:	:60000, column_number:784
test_sample_number:	:10000, column_number:784


NameError: name 'X_testall' is not defined

# preprocessing

In [11]:
X_train=X_train/255
X_test=X_test/255
# X_testall=X_testall/255

# convolution

In [12]:
con = [
    [1,0,-1],
    [1,0,-1],
    [1,0,-1]
]

def convolution(X,con):
    temp_con = np.zeros((26,26))
    for y in range(1,26):
        for x in range(1,26):
            temp = [
                [X[y*28+x-29],X[y*28+x-28],X[y*28+x-27]],
                [X[y*28+x-1],X[y*28+x],X[y*28+x+1]],
                [X[y*28+x+27],X[y*28+x+28],X[y*28+x+29]]
            ]
            mul_=np.array(temp)*np.array(con)
            sum_=np.sum(mul_)
            temp_con[y-1][x-1]=sum_
    return temp_con

# classifer

In [13]:
class myClassifier(object):
    
    def __init__(self, C=1000, eta=0.01, batch_size=60, epochs=200, epsilon=1e-8, shuffle=True, params=None, w=0, b=0):
        self.C = C
        self.eta = eta
        self.batch_size = batch_size
        self.epochs = epochs
        self.epsilon = epsilon
        self.class_num = 0
        self.shuffle = shuffle
        self.update_count = 0
        self.w = 0
        self.b = 0
        
    def fit(self, X, y, params=None, w=0, b=0, testscore = None, eval_score=None):
        
        X_num, X_fea = np.shape(X)
        self.class_num=len(np.unique(y))
        #######################################################
        con_X=np.zeros((X_num,X_fea+(26*26)))
        
        for i in range(X_num):
            con_X[i]=np.append(X[i],convolution(X[i],con))
        
        X=con_X
        #######################################################
        if params is None:
            print('fit params=None')
            self.params = {
                'w': np.random.randn(X_fea+(26*26), self.class_num),
                'b': np.random.randn(1, self.class_num)
            }
        cnt=1
        if eval_score is None:
            self.score_val = 0
                
        for Xi in range(self.epochs):
            s_data, s_labels = self.shuffling(X, y)
            encoded_y=self.encoding(s_labels)
            avg_loss = 0
            batch_count = int(X_num / self.batch_size)
            for t in range(int(batch_count)):
                batch_X, batch_y, bs=self.batching(s_data, encoded_y, t)
                batch_X = np.reshape(batch_X, (bs, X_fea+(26*26)))
                batch_y = np.reshape(batch_y, (bs, self.class_num))
                z = self.net_input(batch_X)
                loss = self.hinge_loss(batch_y, z)
                self.update_w_b(batch_X, batch_y, z, bs, cnt)
                cnt+=1
                avg_loss += loss
                self.update_count += 1
                
            prev_score = self.score_val
            pres_score = self.score(X, y)
            print("epochs: ", Xi)
            print("prev_score: ", prev_score)
            print("pres_score: ", pres_score,"\n")
            if prev_score < pres_score:
                self.score_val = pres_score
            avg_loss /= batch_count
        return self
    
    def update_w_b(self, batch_X, batch_y, z, bs, cnt):
        n = np.shape(batch_X)[1]  # num of features
        delta_w = np.zeros(self.params['w'].shape)
        delta_b = np.zeros(self.params['b'].shape)
        z = np.reshape(z, (bs, self.class_num))
        temp = 1 - np.multiply(batch_y, z)
        temp[temp <= 0] = 0
        temp[temp > 0] = 1
        y_temp = np.multiply(batch_y, temp.reshape(bs, self.class_num))
        delta_w = -(1 / bs) * np.matmul(batch_X.T, y_temp) + (1 / self.C) * self.params['w']
        delta_b = -(1 / bs) * np.sum(y_temp, axis=0)
        self.params['w'] = self.params['w'] - (self.eta / (1 + self.epsilon * cnt)) * delta_w
        self.params['b'] = self.params['b'] - (self.eta / (1 + self.epsilon * cnt)) * delta_b
        
        return self.params
    
    def hinge_loss(self, y, z):
        loss = 1 - np.multiply(y, z)
        loss[loss < 0] = 0
        loss = np.mean(loss)
        return loss
    
    def net_input(self, X):
        z = np.matmul(X, self.params['w']) + self.params['b']
        return z

    def encoding(self, y):
        print(np.shape(y)[0])
        encoded_y=-1*np.ones((np.shape(y)[0],self.class_num))
        for i in range(np.shape(y)[0]):
            encoded_y[i,y[i]] = 1
        return encoded_y
                
    def shuffling(self, X, y):
        temp_s=list(zip(X,y))
        random.shuffle(temp_s)
        X,y=zip(*temp_s)
        return X,y
    
    def batching(self, X, y, t):                         
        batch_X = X[t * self.batch_size : min(len(X), (t+1) * self.batch_size)]
        batch_y = y[t * self.batch_size : min(len(X), (t+1) * self.batch_size)]
        last_size = min(len(X), (t+1) * self.batch_size) - t * self.batch_size
        
        return batch_X, batch_y,last_size
    
    def predict(self, X):
        m = np.shape(X)[0]
        class_score = self.net_input(X)  # return z
        pred = np.argmax(class_score, axis=1)

        return pred
    
    def score(self, X, y):
        pred = self.predict(X)
        score = np.mean(pred == y)
        
        return score
    
    def get_params(self, deep=True):
        return {'C':self.C, 'batch_size':self.batch_size, 'epochs':self.epochs,
               'eta': self.eta, 'w':self.params['w'], 'b':self.params['b']}
    
    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self
    
    def test(self, X, w, b):
        z = np.matmul(X, np.array(w)) + np.array(b)
        p = np.argmax(z, axis=1)
        return p

In [17]:
def shuffling(X, y):
    temp_s=list(zip(X,y))
    random.shuffle(temp_s)
    X,y=zip(*temp_s)
    return X,y
def encoding(y):
    print(type(y))
    print(np.shape(y)[0])
    encoded_y=-1*np.ones((np.shape(y)[0],self.class_num))
    for i in range(np.shape(y)[0]):
        encoded_y[i,y[i]] = 1
    return encoded_y

s_data, s_labels = shuffling(X_train, y_train)
encoded_y=encoding(s_labels)

<class 'tuple'>
60000


NameError: name 'self' is not defined

In [9]:
mine=myClassifier()
mine.fit(X_train,y_train)

(80000, 1460)
fit params=None
epochs:  0
prev_score:  0
pres_score:  0.6412125 

epochs:  1
prev_score:  0.6412125
pres_score:  0.738925 

epochs:  2
prev_score:  0.738925
pres_score:  0.7814625 

epochs:  3
prev_score:  0.7814625
pres_score:  0.80405 

epochs:  4
prev_score:  0.80405
pres_score:  0.819 

epochs:  5
prev_score:  0.819
pres_score:  0.8302 

epochs:  6
prev_score:  0.8302
pres_score:  0.8385125 

epochs:  7
prev_score:  0.8385125
pres_score:  0.8444625 

epochs:  8
prev_score:  0.8444625
pres_score:  0.8492875 

epochs:  9
prev_score:  0.8492875
pres_score:  0.8529625 

epochs:  10
prev_score:  0.8529625
pres_score:  0.856 

epochs:  11
prev_score:  0.856
pres_score:  0.8593625 

epochs:  12
prev_score:  0.8593625
pres_score:  0.862475 

epochs:  13
prev_score:  0.862475
pres_score:  0.8644125 

epochs:  14
prev_score:  0.8644125
pres_score:  0.866875 

epochs:  15
prev_score:  0.866875
pres_score:  0.8685875 

epochs:  16
prev_score:  0.8685875
pres_score:  0.870675 

e

epochs:  141
prev_score:  0.911525
pres_score:  0.9103875 

epochs:  142
prev_score:  0.911525
pres_score:  0.91095 

epochs:  143
prev_score:  0.911525
pres_score:  0.9104 

epochs:  144
prev_score:  0.911525
pres_score:  0.9108 

epochs:  145
prev_score:  0.911525
pres_score:  0.910075 

epochs:  146
prev_score:  0.911525
pres_score:  0.910525 

epochs:  147
prev_score:  0.911525
pres_score:  0.9111125 

epochs:  148
prev_score:  0.911525
pres_score:  0.910575 

epochs:  149
prev_score:  0.911525
pres_score:  0.9102 

epochs:  150
prev_score:  0.911525
pres_score:  0.910225 

epochs:  151
prev_score:  0.911525
pres_score:  0.9094 

epochs:  152
prev_score:  0.911525
pres_score:  0.910225 

epochs:  153
prev_score:  0.911525
pres_score:  0.9099 

epochs:  154
prev_score:  0.911525
pres_score:  0.9103 

epochs:  155
prev_score:  0.911525
pres_score:  0.910175 

epochs:  156
prev_score:  0.911525
pres_score:  0.91125 

epochs:  157
prev_score:  0.911525
pres_score:  0.9114125 

epochs: 

In [ ]:
mine.get_params()

In [ ]:
w=mine.get_params()['w']
b=mine.get_params()['b']

In [ ]:
p=mine.test(X_testall,w,b)

In [ ]:
file=open('/home/ryu/AI/MyClassifier/prediction.txt','w')
for i in range(len(p)):
    file.write('%s\n' %p[i])
file.close()